# SETUP

## Import

In [1]:
import sys
import re
import json
import random
from typing import TypedDict

from shared.constants import *
from shared.corpus import Corpus
from shared.utils import load_json

import spacy
from spacy.language import Language
from spacy.lang.en import English
from spacy.training import Example
from spacy.util import minibatch

# import requests
# from bs4 import BeautifulSoup
# from itertools import chain
# from typing import Any, Iterator

In [2]:
# # Get characters

# url = "https://pemberley.com/janeinfo/ppdrmtis.html"
# res = requests.get(url)
# soup = BeautifulSoup(res.content, "html.parser")

# character_names = []
# target_h2 = soup.find("h2", string="Brief, Organized Listing of Characters")
# target_ul = target_h2.find_next("ul")
# for a in target_ul.find_all("a"):
#     name = a.get_text(strip=True).replace('\n', ' ')
#     if name:
#         character_names.append(name)

# print(character_names)
# with open(JSON_CHARACTERS_PNP, "w", encoding="utf-8") as f:
#     json.dump(character_names, f, indent=4)

# Rule-Based NER (Gazetteer Method)

In [3]:
# Load data
with open(TXT_PNP, "r", encoding="utf-8") as f:
    text = f.read().strip()
    text = re.split(PATTERN_CHAPTER_LINE, text)[1:6]

with open(JSON_CHARACTERS_PNP, "r", encoding="utf-8") as f:
    characters = json.load(f)
    names = set()
    for character in characters:
        names.update(
            [
                i for i in character.split() 
                if not i in ['Mr.', 'The', 'Mrs.', 'Miss', 'Old', 'Lady', 'de', 'Sir', 'Captain', 'Colonel']
            ]
        )
    names = sorted(names)

print(characters)
print(names)

['Mr. Bennet', 'Mrs. Bennet', 'Jane', 'Elizabeth', 'Mary', 'Kitty', 'Lydia', 'Bingley', 'Louisa Hurst', 'Caroline', 'Mr. Collins', 'Old Mr. Darcy', 'Lady Anne Darcy', 'Darcy', 'Georgiana Darcy', 'Lady Catherine', 'Anne de Bourgh', 'Colonel Fitzwilliam', 'Mr. Gardiner', 'Mrs. Gardiner', 'Sir William', 'Lady Lucas', 'Charlotte', 'Maria', 'Old Mr. Wickham', 'Wickham', 'Mrs. Annesley', 'Captain Carter', 'Mr. Chamberlayne', 'Dawson', 'Mr. Denny', 'Colonel Forster', 'William Goulding', 'Miss Grantley', 'Haggerston', 'The Harringtons', 'Mrs. Hill', 'Mr. Hurst', 'Mrs. Jenkinson', 'Mr. Jones', 'Miss Mary King', 'Mrs. Long', 'Lady Metcalfe', 'Mr. Morris', 'Mrs. Nicholls', 'Mr. Philips', 'Miss Pope', 'Mr. Pratt', 'Mrs. Reynolds', 'Mr. Robinson', 'Mr. Stone', 'Miss Watson', 'The Miss Webbs', 'Mrs. Younge']
['Anne', 'Annesley', 'Bennet', 'Bingley', 'Bourgh', 'Caroline', 'Carter', 'Catherine', 'Chamberlayne', 'Charlotte', 'Collins', 'Darcy', 'Dawson', 'Denny', 'Elizabeth', 'Fitzwilliam', 'Forster', 

In [4]:
# Characters by Chapter
for n_chapter, chapter in enumerate(text, start=1):
    print(f'\n\nChapter {n_chapter}:')
    chapter = chapter.strip().replace('\n', ' ').translate(str.maketrans('', '', PUNCTUATION))
    words = chapter.split() # NOTE: Mr and Mrs will be split

    chapter_names = set()
    for n, word in enumerate(words):
        if word in names:
            if (words[n-2] == 'Old') or (words[n-1] == 'de'):
                chapter_names.add(' '.join(words[n-2:n+1])) # Handle 3-word names
            elif words[n-1][0].isupper():
                chapter_names.add(' '.join(words[n-1:n+1])) # Handle 2-word names
            else:
                chapter_names.add(word)
    chapter_names = sorted(chapter_names)
    print(', '.join(chapter_names))



Chapter 1:
Bingley, Jane, Lady Lucas, Lydia, Mr Bennet, Mr Bingley, Mr Morris, Mrs Long, Sir William


Chapter 2:
Elizabeth, Kitty, Lydia, Mary, Mr Bennet, Mr Bingley, Mrs Bennet, Mrs Long, Now Kitty, While Mary


Chapter 3:
Bingley, Catherine, Come Darcy, Elizabeth, Elizabeth Bennet, Jane, Lady Lucas, Lydia, Maria, Maria Lucas, Mary, Miss Bennet, Miss Bingley, Miss King, Miss Lucas, Mr Bennet, Mr Bingley, Mr Darcy, Mr Hurst, Mrs Bennet, Mrs Hurst, Sir William


Chapter 4:
Bingley, Darcy, Elizabeth, Miss Bennet, Miss Bingley, Mr Bingley, Mrs Hurst, When Jane


Chapter 5:
Charlotte, Elizabeth, Jane, Lady Lucas, Lucas, Mary, Miss Bennet, Miss Bingley, Miss Lucas, Mr Darcy, Mr Robinson, Mrs Bennet, Mrs Long, Sir William, William Lucas


# NER with SpaCy

## Data

In [5]:
# Data
hp_corpus = Corpus(DIR_HP)
chapter1 = hp_corpus.books[0].chapters[0].text
chapter1[:100]

'Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly norma'

In [6]:
# url = "https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters"
# res = requests.get(url)
# soup = BeautifulSoup(res.content, "html.parser")

# characters = []
# divs = soup.find_all("div", class_="mw-heading mw-heading3")
# for div in divs:
#     ul = div.find_next("ul")
#     for li in ul.find_all("li"):
#             text = li.get_text(strip=True)
#             name = re.search(r"^(.*?)–", text).group(1).replace(' and ', ',').translate(str.maketrans('()', ',,'))
#             if ',' in name:
#                 split_name = [part.strip() for part in name.split(',') if part.strip()]
#                 last_name = split_name[0]

#                 # Case: <last_name>, <name 1>, <name2>, ...
#                 if len(split_name[1:]) > 1:
#                     for first_name in split_name[1:]:
#                         characters.append(f'{first_name.strip()} {last_name.strip()}')
#                 # Case: <last_name>, <first_name>
#                 else:
#                     characters.append(f'{split_name[1].strip()} {last_name.strip()}')
#             else:
#                 # Case: <name>
#                 characters.append(name.strip())

# print(characters[:10])
# with open(JSON_CHARACTERS_HP, "w", encoding="utf-8") as f:
#     json.dump(characters, f, indent=4)

In [7]:
nlp = spacy.load('en_core_web_lg')

In [8]:
doc = nlp(chapter1)

In [9]:
unique_ents = list({ent.text: ent for ent in doc.ents}.values())
for ent in unique_ents[:10]:
    print(f'{ent.text:<15}{ent.label_:<10}')

Dursley        PERSON    
number four    CARDINAL  
Privet Drive   PERSON    
Grunnings      ORG       
Dursleys       NORP      
Dudley         PERSON    
Potters        ORG       
Potter         PERSON    
several years  DATE      
Tuesday        DATE      


We need to customize the model to make NER for the HP corpus better

# MODELS

## Rule-Based Model (with EntityRuler)

Using EntityRuler pipe we can generate rule-based NER with SpaCy.  
This approach can be used to generate train datasets.

In [10]:
# Generate patterns for characters

def generate_characters(characters: list[str]) -> list:
    """Generate combinations of character names and titles"""
    names = []
    titles = ["Dr.", "Professor", "Mr.", "Mrs.", "Ms.", "Miss", "Aunt", "Uncle", "Mr. and Mrs."]
    
    for item in characters:
        item = item.replace('The', '').replace('the', '').strip()
        names.append(item)
        for name in item.split():
            names.append(name)
    
    generated_characters = set()
    for title in titles:
        for name in names:
            generated_characters.add(name)
            generated_characters.add(f'{title} {name}')
    generated_characters = sorted(generated_characters)
    return generated_characters

def generate_characters_patterns(generated_characters: list) -> list[dict]:
    patterns = []
    for item in generated_characters:
        pattern = {
            "label": "PERSON",
            "pattern": item
        }
        patterns.append(pattern)
    return (patterns)

data: list[str] = load_json(JSON_CHARACTERS_HP)
generated_characters = generate_characters(data)
patterns = generate_characters_patterns(generated_characters)
print(len(patterns))
for pattern in patterns[:5]:
    print(pattern)

4360
{'label': 'PERSON', 'pattern': '"Delphi"'}
{'label': 'PERSON', 'pattern': 'Abbott'}
{'label': 'PERSON', 'pattern': 'Aberforth'}
{'label': 'PERSON', 'pattern': 'Aberforth Dumbledore'}
{'label': 'PERSON', 'pattern': 'Alastor'}


In [11]:
# Create rule-based NER model with EntityRuler pipe

PATH_MODEL_HP_NER_RULE_BASED = os.path.join(DIR_MODELS_SPACY, "hp_ner_rule_based")

def create_rule_based_model(
        patterns: list[dict], 
        path=PATH_MODEL_HP_NER_RULE_BASED
    ) -> None:
    """Create and save a rule-based NER model using custom patterns."""

    # Initialize model with an empty pipeline
    nlp = English() # no pre-trained components (only basic tokenizer included)

    # Add EntityRuler to the model
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)

    # Save model
    nlp.to_disk(path)

def use_rule_based_model(nlp: Language, text: str) -> list[str]:
    """Extract recognized entities."""
    doc = nlp(text)
    return sorted(set(ent.text for ent in doc.ents))

create_rule_based_model(patterns)

In [12]:
# Test model

hp_corpus = Corpus(DIR_HP)
nlp = spacy.load(PATH_MODEL_HP_NER_RULE_BASED)

for n_book, book in enumerate(hp_corpus.books[:1], start=1):
    for n_chapter, chapter in enumerate(book.chapters[:3], start=1): # NOTE: only book 1 chapters 1-3
        print(f'\nBook {n_book}, Chapter {n_chapter}')
        results = use_rule_based_model(nlp, chapter.text)
        results = sorted(set(results))
        print(results)


Book 1, Chapter 1
['Albus', 'Albus Dumbledore', 'Dedalus Diggle', 'Dudley', 'Dumbledore', 'Godric', 'Hagrid', 'Harry', 'Harry Potter', 'James', 'James Potter', 'Lily', 'Madam', 'Mr. Dursley', 'Mr. and Mrs. Dursley', 'Mrs. Dursley', 'Mrs. Potter', 'Muggle', 'Petunia', 'Pomfrey', 'Potter', 'Professor Dumbledore', 'Professor McGonagall', 'Sirius', 'Sirius Black', 'Ted', 'Voldemort']

Book 1, Chapter 2
['Aunt Petunia', 'Dudley', 'Dudley Dursley', 'Harry', 'Harry Potter', 'Marge', 'Mr. Dursley', 'Mrs. Figg', 'Nearly', 'Uncle Vernon', 'Vernon']

Book 1, Chapter 3
['Aunt Petunia', 'Dudley', 'Harry', 'Marge', 'Mrs. Figg', 'Nearly', 'Petunia', 'Potter', 'Uncle Vernon', 'Vernon']


There are still many unrecognized entities because a hardcoded list can never be complete (e.g. "Ronald" was not in the original list), and typos can occur. However, this approach may help generate training data.

### Generate training data using rule-based NER

We generate weakly supervised training data using rule-based NER adn then use this data for training spaCy model. 


Train data structure:
```json
[
  [
    "<text>",
    {
      "entities":
        [
          [<start>, <end>, "<label>"], 
          ...
        ]
    }
  ],
  ...
]
```

In [13]:
# Data types for type and field hints
TrainDataEntity = tuple[int, int, str]
TrainDataItem = tuple[str, dict[str, list[TrainDataEntity]]] # (text, annotation)

In [14]:
def generate_train_data_item(model: Language, text: str) -> TrainDataItem:
    doc = model(text)
    entities = []
    for ent in doc.ents:
        entity: TrainDataEntity = (ent.start_char, ent.end_char, ent.label_)
        entities.append(entity)
    annotation = {
        "entities": entities
    }
    data_item: TrainDataItem = (text, annotation)
    return data_item

nlp = spacy.load(PATH_MODEL_HP_NER_RULE_BASED)

example_entity = generate_train_data_item(nlp, chapter1[:30])
print(example_entity)
print(chapter1[0:20])

('Mr. and Mrs. Dursley, of numbe', {'entities': [(0, 20, 'PERSON')]})
Mr. and Mrs. Dursley


In [15]:
def generate_train_data(model: Language, text: str):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(text)
    for sent in doc.sents:
        data_item = generate_train_data_item(model, sent.text)
        if data_item[1]["entities"]:
            yield data_item # only data with found entities

In [16]:
# # Generate and save train data

# def dump_to_json(generator: Iterator[TrainDataItem], filename: str, chunk_size=1000) -> None:
#     with open(filename, 'w', encoding='utf-8') as f:
#         f.write('[')
#         item = next(generator)
#         json.dump(item, f, ensure_ascii=False) # write 1st item

#         batch = []
#         for item in generator:
#             batch.append(item)
#             if len(batch) >= chunk_size:
#                 f.write(',\n')
#                 items = (json.dumps(item, ensure_ascii=False) for item in batch)
#                 f.write(',\n'.join(items))
#                 f.flush()
#                 batch.clear()
        
#         if batch:
#             f.write(',\n')
#             items = (json.dumps(batch_item, ensure_ascii=False) for batch_item in batch)
#             f.write(',\n'.join(items))
#         f.write('\n]')

# hp_rule_based_ner_model = spacy.load(path_hp_rule_based_ner_model)
# en_core_web_lg = spacy.load("en_core_web_lg")

# generators = []
# for book in hp_corpus.books:
#     for chapter in book.chapters:
#         generators.append(generate_train_data(hp_rule_based_ner_model, chapter.text))
        
# dump_to_json(chain.from_iterable(generators), JSON_TRAIN_DATA_HP)

In [17]:
TRAIN_DATA = load_json(JSON_TRAIN_DATA_HP)
for i in TRAIN_DATA[:3]:
    print(i)
print(len(TRAIN_DATA))

['Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.', {'entities': [[0, 20, 'PERSON']]}]
['Mr. Dursley was the director of a firm called Grunnings, which made drills.', {'entities': [[0, 11, 'PERSON']]}]
['Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.', {'entities': [[0, 12, 'PERSON']]}]
43607


## Train spaCy Model

In [18]:
def train_spacy_model(
        train_data: list[TrainDataItem], 
        EPOCHS: int = 30,
        BATCH_SIZE: int = 8,
        DROPOUT: float = 0.2
    ):

    # Create empty English pipeline
    nlp = spacy.blank("en")

    # Create/get NER component
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")
    
    # Add labels to NER component
    for text, annotation in train_data:
        for ent in annotation["entities"]:
            ner.add_label(ent[2])

    # Train NER component
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.select_pipes(disable=other_pipes): # train only NER component
        optimizer = nlp.begin_training()

        examples = [
            Example.from_dict(
                nlp.make_doc(text), 
                annotation
            ) 
            for text, annotation in train_data
        ]
        for epoch in range(EPOCHS):
            print("Epoch", epoch, end=" --- ")
            random.shuffle(TRAIN_DATA) # ensure that the model doesn't just memorize the order
            losses = {}

            for batch in minibatch(examples, size=BATCH_SIZE):
                nlp.update(batch, sgd=optimizer, drop=DROPOUT, losses=losses)
            loss = losses['ner']
            print('Loss:', loss)
            
            if loss < 10: # NOTE: Primitive early stopping 
                print(f'Stopping early at epoch {epoch}')
                break
            
    return nlp

In [22]:
# Train model

# %%capture captured_output
sys.exit("Manual execution required");

TRAIN_DATA = load_json(JSON_TRAIN_DATA_HP)[:3000]
nlp = train_spacy_model(TRAIN_DATA, EPOCHS=30)

PATH_MODEL_HP_NER_TRAINED_SIMPLE = os.path.join(DIR_MODELS_SPACY, 'hp_ner_trained_simple')
nlp.to_disk(PATH_MODEL_HP_NER_TRAINED_SIMPLE)

SystemExit: Manual execution required

/home/asdf/Desktop/nlp-basics/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
def find_test_chapter(corpus: Corpus, substr: str) -> tuple[int, int] | None:
    for n_book, book in enumerate(corpus.books):
        for n_chapter, chapter in enumerate(book.chapters):
            if substr in chapter.text:
                print(f'Use Book {n_book+1}, Chapter {n_chapter+1}')
                return (n_book, n_chapter)
            
book_idx, chapter_idx = find_test_chapter(hp_corpus, 'Ronald') # type: ignore

Use Book 1, Chapter 12


In [24]:
# Compare trained and rule-based models

hp_corpus = Corpus(DIR_HP)
text = hp_corpus.books[book_idx].chapters[chapter_idx].text

PATH_MODEL_HP_NER_RULE_BASED = os.path.join(DIR_MODELS_SPACY, 'hp_ner_rule_based')
PATH_MODEL_HP_NER_TRAINED_SIMPLE = os.path.join(DIR_MODELS_SPACY, 'hp_ner_trained_simple')
hp_model_ner_rule_based = spacy.load(PATH_MODEL_HP_NER_RULE_BASED)
hp_model_ner_trained_simple = spacy.load(PATH_MODEL_HP_NER_TRAINED_SIMPLE)
results_rule_based = set()
results_trained = set()

for model, results in [
        (hp_model_ner_rule_based, results_rule_based), 
        (hp_model_ner_trained_simple, results_trained)
    ]:
    doc = model(text)
    unique_names = sorted({ent.text for ent in doc.ents})
    for ent in doc.ents:
        results.add(ent.text)

In [25]:
print(f"Difference (rule_based - trained): {sorted(results_rule_based-results_trained)}")
print(f"Difference (trained - rule_based): {sorted(results_trained-results_rule_based)}")

Difference (rule_based - trained): []
Difference (trained - rule_based): ['Forge', 'Quick']


It still doesn't find 'Ronald'.  
The model can be significantly improved with Word Embedding vectors.